# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорт необходимых библиотек

In [15]:
import pandas as pd
import numpy as np
import nltk
import re
import torch
import transformers

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [13]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Подготовка данных

In [4]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
print(df.head(5))
print('-' * 30)
print('Дубликаты:', df.duplicated().sum())
print('-' * 30)
print('Пропуски:', df.isna().sum())
print('-' * 30)
print('Соотношение в целевом признаке:', df.toxic.value_counts(normalize=True))

   Unnamed: 0                                               text  toxic
0           0  Explanation\nWhy the edits made under my usern...      0
1           1  D'aww! He matches this background colour I'm s...      0
2           2  Hey man, I'm really not trying to edit war. It...      0
3           3  "\nMore\nI can't make any real suggestions on ...      0
4           4  You, sir, are my hero. Any chance you remember...      0
------------------------------
Дубликаты: 0
------------------------------
Пропуски: Unnamed: 0    0
text          0
toxic         0
dtype: int64
------------------------------
Соотношение в целевом признаке: 0    0.898388
1    0.101612
Name: toxic, dtype: float64


Нужно избавиться от столбца `Unnamed`, так как данный столбец дублирует индексацию и не важен в обучении модели.

In [6]:
df = df.drop(['Unnamed: 0'], axis=1)

Лемматизируем и очистим датасет от лишних символов.

In [7]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
def clear_text(text: str) -> str:
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return re_list

m = WordNetLemmatizer()

def lemmatize_text(text: str) -> str:
    word_list = nltk.word_tokenize(text)
    return ' '.join([m.lemmatize(w, get_wordnet_pos(w)) for w in word_list])

In [11]:
df['text'] = df['text'].apply(clear_text)

In [16]:
df['lemm_text'] = df['text'].apply(lemmatize_text)

In [17]:
df

,text,toxic,lemm_text
0,Explanation Why the edits made under my userna...,0,Explanation Why the edits make under my userna...
1,D'aww He matches this background colour I'm se...,0,D'aww He match this background colour I 'm see...
2,Hey man I'm really not trying to edit war It's...,0,Hey man I 'm really not try to edit war It 's ...
3,More I can't make any real suggestions on impr...,0,More I ca n't make any real suggestion on impr...
4,You sir are my hero Any chance you remember wh...,0,You sir be my hero Any chance you remember wha...
...,...,...,...
159287,And for the second time of asking when your vi...,0,And for the second time of ask when your view ...
159288,You should be ashamed of yourself That is a ho...,0,You should be ashamed of yourself That be a ho...
159289,Spitzer Umm theres no actual article for prost...,0,Spitzer Umm there no actual article for prosti...
159290,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...


Удалим столбец `text`, так как мы его не будем использовать в обучении моделей.

In [18]:
df = df.drop(['text'], axis=1)

Создадим корпус из лемматизированных и очищенных текстов

In [19]:
corpus = df['lemm_text'].values

Выполним сплитование

In [20]:
features = corpus
target = df['toxic'].values

train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=12345)

In [21]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
train_features = count_tf_idf.fit_transform(train_features)
test_features = count_tf_idf.transform(test_features)

**Промежуточный вывод:**

- Был удалён столбец `Unnamed`, так как он дублировал индексирование датафрейма
- Были очищены и лемматизированы данных из столбца `text`. Итоговые даннные были помещены в столбец `lemm_text`
- Столбец `text` был удалён за ненадобностью, так как все необходимые данные хранятся в `lemm_text`
- Данные были разделены на тренировочную и тестовую выборки
- Тренировочные данные были векторизованы с помощью `TfidfVectorizer`

## Обучение

### Логистическая регрессия

In [22]:
params = {
    'class_weight': ['balanced'],
    'C': [10]
}

model_lr = LogisticRegression()
grid_lr = GridSearchCV(model_lr, params, scoring='f1', cv=3)
grid_lr.fit(train_features, train_target)

print('Итоговое значение F1-score:', grid_lr.best_score_)

Итоговое значение F1-score: 0.7550162516015498


### CatBoostClassifier

In [23]:
#train_features_cat =  train_features.toarray() 

model_cat = CatBoostClassifier(eval_metric='F1', iterations=100, max_depth=10, learning_rate=0.9, random_state=12345)
model_cat.fit(train_features, train_target, verbose=20)

print('Итоговое значение F1-score:', model_cat.best_score_['learn']['F1'])

0:	learn: 0.4198349	total: 21s	remaining: 34m 41s
20:	learn: 0.7577993	total: 8m 6s	remaining: 30m 29s
40:	learn: 0.7911395	total: 16m 16s	remaining: 23m 25s
60:	learn: 0.8119662	total: 23m 47s	remaining: 15m 12s
80:	learn: 0.8299983	total: 32m 13s	remaining: 7m 33s
99:	learn: 0.8429580	total: 39m 28s	remaining: 0us
Итоговое значение F1-score: 0.8429579523153368


### LightBMClassifier

In [ ]:
params = {
    'max_depth': range(1, 13, 3),
    'learning_rate': np.arange(0.1, 1.1, 0.3),
    'n_estimators': range(51, 101, 10)
}

model_lgbm = LGBMClassifier()

grid_lgbm = GridSearchCV(model_lgbm, params, cv=3, scoring='f1')
grid_lgbm.fit(train_features, train_target)

print('Итоговое значение F1-score:', grid_lgbm.best_score_)

Систематизируем полученные данные для определения лучшей модели

In [ ]:
data = {
    'LogisticRegression': grid_lr.best_score_,
    'CatBosstClassifier': model_cat.best_score_['learn']['F1'],
    'LightGBMClassifier': grid_lgbm.best_score_
}

df_results = pd.DataFrame(
    data=data,
    index=['f1_score'],
)
df_results.T

Исходя из полученных значений на тренировочных данных, можно сделать вывод, что лучшей моделью для заказчика будет `CatBoostClassifier` с `f1-score` в **0.846685**. Проверим данную модель на тестовых данных.

In [ ]:
predictions = model_cat.predict(test_features)
print('Итоговое значение f1:', f1_score(test_target, predictions))

Значение `f1-score` на тестовых данных получилось **0.767939715704744**, что больше **0.75**, значит, условия задачи были выполнены.

## Итоговый вывод

**Предобработка данных:**

- Был удалён столбец `Unnamed`, так как он дублировал индексирование датафрейма
- Были очищены и лемматизированы данных из столбца `text`. Итоговые даннные были помещены в столбец `lemm_text`
- Столбец `text` был удалён за ненадобностью, так как все необходимые данные хранятся в `lemm_text`
- Данные были разделены на тренировочную и тестовую выборки
- Тренировочные данные были векторизованы с помощью `TfidfVectorizer`

**Обучение моделей:**
- Исходя из полученных значений на тренировочных данных, можно сделать вывод, что лучшей моделью для заказчика будет `CatBoostClassifier` с `f1-score` в **0.846685**.
- значение `f1-score` на тестовых данных модели `CatBoostClassifier` получилось **0.767939715704744**, что больше **0.75**, значит, условия задачи были выполнены.